# Machine Learning - Master IEF Parcours Quantitatif
## Prediction of Sharpe ratio for blends of quantitative strategies par Napoleon X (2019)
### Giovanni MANCHE et Antonin DEVALLAND

Lien vers le GitHub du projet : https://github.com/GiovanniManche/MachineLearning272

## Table des matières
J'arrive pas à la mettre   + faudra suivre l'outline donnée dans Moodle

## I/ Sujet

L'objectif de ce problème est d'aider à la construction d'un **mélange optimal de stratégies quantitatives d'investissement** à partir d'un ensemble de stratégies. Trouver la meilleure allocation parmi les stratégies quantitatives chaque semaine revient à déterminer la combinaison maximisant le ratio de Sharpe (sur les 5 prochains jours de trading). Avec $Lr_{i,s}$ les rendements logarithmiques d'une stratégie $i$ à un temps $s$, le ratio de Sharpe annualisé objectif est calculé selon la formule suivante (il est légèrement modifié pour éviter une volatilité trop proche de 0) : 
$$
S_t^*(w_1, \ldots, w_7) = \frac{\frac{252}{5} \sum_{i=1}^{7} w_i \times \left( \sum_{s=t+1}^{t+5} Lr_{i,s} \right)}{Max \left( \sqrt{252 \times \sum_{i=1}^{7} \sum_{j=1}^{7} w_i w_j \sum_{s=t-20}^{t+5} (Lr_{i,s} - \bar{Lr_i})(Lr_{j,s} - \bar{Lr_j})} ; 0.005 \right)}
$$

L'enjeu est donc, finalement, de prédire le ratio de Sharpe $S^*$ d'une combinaison donnée de poids et d'optimiser en tenant compte du compromis rendement / volatilité. 

## Import des bibliothèques nécessaires 

In [ ]:
import pandas as pd

## Partie I : Récupération et préparation des données

Le fichier CSV fournit pour l'entraînement pesait trop lourd (plus de 35 000 Ko, alors que la limite pour déposer les données sur GitHub est de 25 000 Ko). Nous l'avons donc converti en un fichier xlsx., moins lourd, au prix d'un temps d'importation via Python allongé.

In [ ]:
train_set: pd.DataFrame = pd.read_excel("Data/Train set.xlsx")
test_set: pd.DataFrame = pd.read_csv("Data/Test set.csv")
target_values: pd.DataFrame = pd.read_csv("Data/Target values.csv")

Nous effectuons quelques vérifications pour s'assurer que le jeu de données est utilisable : 
- gestion des données manquantes
- gestion des données dupliquées
- uniformisation des formats de données
- gestion des outliers

Tout d'abord, on s'assure que les fichiers ne présentent pas de lignes dupliquées.

In [ ]:
print(f"Les dimensions du train set initial sont {train_set.shape} et celles du nouveau train set sont de {train_set.drop_duplicates().shape}.")
print(f"Les dimensions du train set initial sont {test_set.shape} et celles du nouveau train set sont de {test_set.drop_duplicates().shape}.")
print(f"Les dimensions du train set initial sont {target_values.shape} et celles du nouveau train set sont de {target_values.drop_duplicates().shape}.")

Les dimensions du train set initial sont (10000, 218) et celles du nouveau train set sont de (10000, 218)
Les dimensions du train set initial sont (4450, 218) et celles du nouveau train set sont de (4450, 218)
Les dimensions du train set initial sont (10000, 2) et celles du nouveau train set sont de (10000, 2)


Ensuite, on vérifie l'absence de données manquantes.

In [10]:
print(f"Le fichier contenant les données d'entraînement contient {train_set.isnull().sum().sum()} données manquantes.")
print(f"Le fichier contenant les données de test contient {test_set.isnull().sum().sum()} données manquantes.")
print(f"Le fichier contenant les valeurs cibles contient {target_values.isnull().sum().sum()} données manquantes.")

Le fichier contenant les données d'entraînement contient 0 données manquantes.
Le fichier contenant les données de test contient 0 données manquantes.
Le fichier contenant les valeurs cibles contient 0 données manquantes.


À toutes fins utiles, on s'assure que nos données sont toutes de type float. En effet, le train set et test set sont composés de valeurs décimales (parts, valeurs des stratégies, valeurs des instruments financiers), il n'y a donc pas de raison qu'elles soient autre chose que de type float.

In [15]:
train_set = train_set.astype(float)
target_values = target_values.astype(float)
test_set = test_set.astype(float)

In [23]:
train_set

,ID,weight_I_1,weight_I_2,weight_I_3,weight_I_4,weight_I_5,weight_I_6,weight_I_7,I_1_lag_20,I_1_lag_19,...,X_3_lag_9,X_3_lag_8,X_3_lag_7,X_3_lag_6,X_3_lag_5,X_3_lag_4,X_3_lag_3,X_3_lag_2,X_3_lag_1,X_3_lag_0
0,0.0,0.15,0.00,0.05,0.80,0.00,0.00,0.00,100.0,100.047398,...,101.383783,102.054669,102.375596,103.148605,103.148605,103.046483,103.075701,103.134043,103.221509,103.338192
1,1.0,0.00,0.00,0.00,0.40,0.25,0.00,0.35,100.0,99.912339,...,100.911142,100.938707,100.993926,101.132016,100.745489,100.524617,100.303743,100.276090,100.303743,100.554527
2,2.0,0.85,0.00,0.00,0.15,0.00,0.00,0.00,100.0,99.481681,...,100.373084,100.581716,100.313489,100.790251,101.013756,100.686030,100.686030,100.060233,99.747384,99.970889
3,3.0,0.00,0.00,0.70,0.05,0.25,0.00,0.00,100.0,100.124618,...,100.844136,101.040072,101.055122,101.567682,101.703322,101.974603,101.733422,101.838963,102.080144,101.688272
4,4.0,0.00,0.55,0.05,0.00,0.00,0.00,0.40,100.0,100.000000,...,99.665093,99.482389,99.604192,100.030499,99.847797,100.426310,100.426310,100.822217,100.913521,100.852619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995.0,0.50,0.00,0.00,0.45,0.00,0.00,0.05,100.0,100.133669,...,100.912980,101.768901,101.825961,102.282452,103.280979,102.738943,102.510698,102.596243,103.280979,103.223917
9996,9996.0,0.50,0.00,0.15,0.35,0.00,0.00,0.00,100.0,100.830429,...,101.541742,101.197305,101.771363,102.345411,102.657045,102.919468,102.804653,102.870260,102.952272,102.870260
9997,9997.0,0.10,0.00,0.00,0.00,0.00,0.65,0.25,100.0,100.000000,...,98.496241,98.527571,97.744362,98.088972,98.057644,98.214286,97.525063,97.274436,96.773184,96.804512
9998,9998.0,0.00,0.00,0.00,0.15,0.15,0.00,0.70,100.0,99.985526,...,102.002301,102.017204,102.402799,102.210001,102.106150,101.438717,101.928162,102.032013,102.076533,102.402799


On peut désormais faire quelques vérifications sur les valeurs : 
- les valeurs des parts doivent être comprises entre 0 et 1 (répartition de l'AUM entre différentes stratégies)
- la somme des parts doit être égale à 1 (tolérance de $10^{-4}$)
- les valeurs initiales des stratégies et des instruments financiers doivent être égales à 100 (initialisation)


In [ ]:
# Tests sur les poids
weights_train = train_set.iloc[:,1:8]
weights_test = test_set.iloc[:, 1:8]
# Ensemble d'entraînement
print(f"Dans le set d'entraînement, {((weights_train < 0) | (weights_train > 1)).sum().sum()} samples présentent de poids supérieurs à 1 ou inférieurs à 0")
print(f"Dans le set d'entraînement, {abs((weights_train.sum(axis=1) - 1)> 0.0001).sum()} samples ont une somme totale de poids différents de 1")
# Ensemble de test
print(f"Dans le set de test, {((weights_test < 0) | (weights_test > 1)).sum().sum()} samples présentent de poids supérieurs à 1 ou inférieurs à 0")
print(f"Dans le set de test, {abs((weights_test.sum(axis=1) - 1)> 0.0001).sum()} samples ont une somme totale de poids différents de 1")


Dans le set d'entraînement, 0 samples présentent de poids supérieurs à 1 ou inférieurs à 0
Dans le set d'entraînement, 0 samples ont une somme totale de poids différents de 1
Dans le set de test, 0 samples présentent de poids supérieurs à 1 ou inférieurs à 0
Dans le set de test, 0 samples ont une somme totale de poids différents de 1


In [64]:
# Tests sur les valeurs initiales
init_train = train_set.filter(like = 'lag_20')
init_test = test_set.filter(like = 'lag_20')
print(f"Dans le set d'entraînement, {((init_train != 100)).sum().sum()} samples présentent des valeurs initiales de portefeuilles ou d'instruments différentes de 100")
print(f"Dans le set de test, {((init_test != 100)).sum().sum()} samples présentent des valeurs initiales de portefeuilles ou d'instruments différentes de 100")

Dans le set d'entraînement, 0 samples présentent des valeurs initiales de portefeuilles ou d'instruments différentes de 100
Dans le set de test, 0 samples présentent des valeurs initiales de portefeuilles ou d'instruments différentes de 100


## Analyse exploratoire